In [ ]:
import sys
sys.path.append('path/to/UAVTrafficPy')

In [25]:
from UAVTrafficPy.tests import dataload_example
from UAVTrafficPy.tool import uavtrafficpy
tool=uavtrafficpy.Master()

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
dataset_information={
    'day':'24Oct',
    'month':'10',
    'drone':'2',
    'time_period':'0900_0930',
    'road':'Panepistimiou',
    'intersection':'Panepistimiou-Omirou'
    }

nickname = f'Drone{dataset_information.get('drone')}_{dataset_information.get('day')}_{dataset_information.get('time_period')}'
file_path= fr'C:\Users\kpourg01\Desktop\Work\Code\pNEUMA\datasets\{dataset_information.get('road')}/2018{dataset_information.get('month')}{dataset_information.get('day')[:2]}_d{dataset_information.get('drone')}_{dataset_information.get('time_period')}.csv'

In [ ]:
#sampling period must be integer multiple of 0.04
period_dataloading=0.2
time_decimals=1
#===========================================================================================
raw_data = dataload_example.get_info_pneuma(file_path=file_path, sampling_period=period_dataloading, time_decimals=time_decimals, position_smoothing=2)

In [ ]:
period_flowcount=1
#----------------------------------------------------
start = min(min(set) for set in raw_data.get('time'))
end = max(max(set) for set in raw_data.get('time'))
time_axis = np.round(np.arange(start,end+period_flowcount,period_flowcount),decimals=time_decimals).tolist()
#----------------------------------------------------
ll_lat,ll_lon = 37.97811671602297, 23.733975874806358
lr_lat,lr_lon = 37.97876143771719, 23.735210640421347
ur_lat,ur_lon = 37.97938790250702, 23.734674127747873
ul_lat,ul_lon = 37.97874587304865, 23.73345028699124
#----------------------------------------------------
bbox = [(ll_lat,ll_lon), 
        (lr_lat,lr_lon),
        (ur_lat,ur_lon),
        (ul_lat,ul_lon)]
#----------------------------------------------------
intersection_center=(37.97866950849114, 23.734362398006162)
#----------------------------------------------------
spatio_temporal_info =  {'bbox':bbox,'intersection center':intersection_center,'time axis': time_axis}

In [ ]:
dataloader = tool.dataloader(raw_data,spatio_temporal_info)
data = dataloader.get_filtered_data(cursed_ids=[208,1442,75,2450,140,1151,1469,628,2941,154,917,614,682,326,1246,1281,1817,2021,1659,2480,2509])
analysis = tool.analysis(data,spatio_temporal_info)
visualization = tool.visualization(data,spatio_temporal_info)

In [ ]:
visualization.draw_trajectories_od()

In [ ]:
od_pairs = analysis.get_od_pairs()
valid_od_pairs = [(1,3),(1,2),(4,3),(4,2)]
visualization.draw_trajectories_cc(valid_od_pairs)

In [ ]:
fig,ax = plt.subplots(nrows=1,ncols=2,figsize=(10,3))

labels_a = [r'1$\to$3', r'1$\to$2']
sizes_a = [100*sum([1 for pair in od_pairs if pair in [(1,3)]])/(len(od_pairs)), 100*sum([1 for pair in od_pairs if pair in [(1,2)]])/(len(od_pairs))]

#ax[0].set_title('Turn ratios for trajectories originating from Panepistimiou Ave.')
ax[0].pie(sizes_a,labels=labels_a,colors=['blue','orange'],autopct='%1.1f%%', textprops={'fontsize':13,'fontweight':'bold'},startangle=90)

labels_b = [r'4$\to$2',r'4$\to$3']
sizes_b = [100*sum([1 for pair in od_pairs if pair in [(4,2)]])/(len(od_pairs)), 100*sum([1 for pair in od_pairs if pair in [(4,3)]])/(len(od_pairs))]

#ax[1].set_title('Turn ratios for trajectories originating from Omirou Str.')
ax[1].pie(sizes_b,labels=labels_b,colors=['green','red'],autopct='%1.1f%%', textprops={'fontsize':13,'fontweight':'bold'},  startangle=90)


plt.tight_layout()
plt.show(close=True)

In [ ]:
data_a = analysis.get_od_data(desirable_pairs=[(1,3),(1,2)])
analysis_a = tool.analysis(data_a,spatio_temporal_info)
visual_a = tool.visualization(data_a,spatio_temporal_info)
#----------------------------------------------------------------------
data_b = analysis.get_od_data(desirable_pairs=[(4,3),(4,2)])
analysis_b = tool.analysis(data_b,spatio_temporal_info)
visual_b = tool.visualization(data_b,spatio_temporal_info)

In [ ]:
lane_info_a = analysis_a.get_lane_info(flow_direction='up',nbins=100,valid_od_pairs=[(1,3)])

In [ ]:
lane_info_b = analysis_b.get_lane_info(flow_direction='left',nbins=3,valid_od_pairs=[(4,2)])

In [ ]:
detector_positions_a = (37.978572395827086, 23.734345599162733)
#---------------------------
flow_info_a = analysis_a.get_flow_info(detector_positions_a)
flow_a,norm_flow_a = analysis_a.get_normalized_flow(threshold=10)
#============================
detector_positions_b = (37.97876743300198, 23.734464957457945)
#----------------------------
flow_info_b = analysis_b.get_flow_info(detector_positions_b)
flow_b,norm_flow_b = analysis_b.get_normalized_flow(threshold=15)

In [ ]:
print(f'flow counts main: {sum(flow_a)}, max vehicles: {len(data_a.get('id'))}')
print(f'\nflow counts secondary: {sum(flow_b)}, max vehicles: {len(data_b.get('id'))}')

In [ ]:
legend_a=r'1 $\to$ 3,2'
legend_b=r'4 $\to$ 3,2'
visualization.draw_traffic_light_phases(legend_a,legend_b,norm_flow_a,norm_flow_b,flow_a,flow_b)

In [ ]:
tlp_a = analysis_a.get_traffic_light_phases()
tlp_b = analysis_b.get_traffic_light_phases()
#==================================================
cycles = analysis.get_traffic_light_cycles(tlp_a,tlp_b)

In [ ]:
sorted_id_a = analysis_a.get_sorted_id()
gaps_a = analysis_a.get_gaps()
#==============================================
sorted_id_b = analysis_b.get_sorted_id()
gaps_b = analysis_b.get_gaps()

In [ ]:
queue_info_a = analysis_a.get_queue_info(speed_threshold=10, gap_threshold=7)
queue_info_b = analysis_b.get_queue_info(speed_threshold=10, gap_threshold=7)